In [1]:
def my_dec(a,b):
    def dec(fn):
        def inner(*args,**kwargs):
            print('decorated function called: a={0}, b={1}'.format(a,b))
            return fn(*args,**kwargs)
        return inner
    return dec

In [2]:
@my_dec(10,20)
def my_func(s):
    print('Hello {0}'.format(s))

In [3]:
my_func('World')

decorated function called: a=10, b=20
Hello World


In [4]:
class MyClass:
    def __init__(self,a,b):
        self.a = a
        self.b = b
        
    def __call__(self,c):
        print('called a={0}, b={1},c={2}'.format(self.a,self.b,c))

In [5]:
obj = MyClass(10,20)

In [6]:
obj

In [7]:
obj.__call__(100)

called a=10, b=20,c=100


In [8]:
obj(100)

called a=10, b=20,c=100


In [9]:
class MyClass:
    def __init__(self,a,b):
        self.a = a
        self.b = b
        
    def __call__(self,fn):
        def inner(*args,**kwargs):
            print('decorated function called: a={0}, b={1}'.format(self.a,self.b))
            return fn(*args,**kwargs)
        return inner

In [10]:
@MyClass(10,20)
def my_func(s):
    print('Hello {0}'.format(s))

In [11]:
my_func('World')

decorated function called: a=10, b=20
Hello World


In [12]:
obj = MyClass(10,20)

In [13]:
def my_func(s):
    print('Hello {0}'.format(s))

In [14]:
my_func = obj(my_func)

In [15]:
my_func('World')

decorated function called: a=10, b=20
Hello World


## Decorating Classes

In [16]:
from fractions import Fraction

In [17]:
f = Fraction(2,3)

In [18]:
f.numerator

2

In [19]:
#Monkey Patching
Fraction.is_integral = lambda self:self.denominator == 1

In [20]:
f1 = Fraction(2,3)
f2 = Fraction(64,8)

In [21]:
f1

Fraction(2, 3)

In [22]:
f2

Fraction(8, 1)

In [23]:
f1.is_integral()

False

In [24]:
f2.is_integral()

True

In [25]:
def dec_speak(cls):
    cls.speak = lambda self, message: f'{self.__class__.__name__} says {message}'
    return cls

In [26]:
Fraction = dec_speak(Fraction)

In [27]:
f1 = Fraction(2,3)
f1.speak('hello')

'Fraction says hello'

In [28]:
class Person:
    pass

In [29]:
Person = dec_speak(Person)

In [30]:
p = Person()

In [31]:
p.speak('this works!')

'Person says this works!'

In [32]:
from datetime import datetime, timezone

In [33]:
def info(self):
    results = []
    results.append(f'time: {datetime.now(timezone.utc)}')
    results.append(f'Class: {self.__class__.__name__}')
    results.append('id: {0}'.format(hex(id(self))))
    for k,v in vars(self).items():
        results.append(f'{k}:{v}')
    return results

def debug_info(cls):
    cls.debug = info
    return cls

In [34]:
@debug_info
class Person:
    def __init__(self,name):
        self.name = name

In [37]:
p = Person('John')

In [38]:
p.debug()

['time: 2019-11-28 18:41:39.661558+00:00',
 'Class: Person',
 'id: 0x10b3789b0',
 'name:John']

In [42]:
@debug_info
class Automobile:
    def __init__(self,make,model,year,top_speed):
        self.make = make
        self.model = model
        self.year = year
        self.top_speed = top_speed
        self._speed = 0
        
    @property
    def speed(self):
        return self._speed
    
    @speed.setter
    def speed(self,new_speed):
        if new_speed > self.top_speed:
            raise ValueError('Speed cannot exceed top_speed')
        else:
            self._speed = new_speed

In [43]:
favorite = Automobile('Ford','Model T',1908,45)

In [44]:
favorite.debug()

['time: 2019-11-28 18:51:49.221840+00:00',
 'Class: Automobile',
 'id: 0x10b38d898',
 'make:Ford',
 'model:Model T',
 'year:1908',
 'top_speed:45',
 '_speed:0']

In [46]:
favorite.speed = 40

In [47]:
favorite.debug()

['time: 2019-11-28 18:52:31.315258+00:00',
 'Class: Automobile',
 'id: 0x10b38d898',
 'make:Ford',
 'model:Model T',
 'year:1908',
 'top_speed:45',
 '_speed:40']

In [48]:
from math import sqrt

In [50]:
class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    
    def __abs__(self):
        return sqrt(self.x**2 +self.y**2)
    
    def __repr__(self):
        return f'Point({self.x},{self.y})'

In [51]:
p1,p2,p3 = Point(2,3),Point(2,3),Point(0,0)

In [52]:
abs(p1)

3.605551275463989

In [60]:
class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    
    def __abs__(self):
        return sqrt(self.x**2 +self.y**2)
    
    def __repr__(self):
        return f'Point({self.x},{self.y})'
    
    def __eq_(self,other):
        if isinstance(other,Point):
            return self.x == other.x and self.y == other.y
        else:
            return False
        
    def __lt__(self,other):
        if isinstance(other,Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented

In [61]:
p1,p2,p3 = Point(2,3),Point(2,3),Point(0,0)

In [62]:
p1 == p2

False

In [63]:
p3 < p1

True

In [64]:
p4 = Point(100,100)

In [65]:
p4 < p1

False

In [66]:
p4 > p1

True

- a <= b iff a < b or a == b
- a > b  iff not(a < b) and a != b
- a >= b  iff not(a > b)

In [67]:
def complete_ordering(cls):
    if '__eq__' in dir(cls) and '__lt__' in dir(cls):
        cls.__le__= lambda self,other: self < other and self == other
        cls.__gt__ = lambda self, other: not (self < other) and not (self == other)
        cls.__ge__ = lambda self,other: not(self < other)
    return cls

In [69]:
@complete_ordering
class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    
    def __abs__(self):
        return sqrt(self.x**2 +self.y**2)
    
    def __repr__(self):
        return f'Point({self.x},{self.y})'
    
    def __eq_(self,other):
        if isinstance(other,Point):
            return self.x == other.x and self.y == other.y
        else:
            return False
        
    def __lt__(self,other):
        if isinstance(other,Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented

In [70]:
p1,p2,p3,p4 = Point(2,3),Point(2,3),Point(0,0),Point(100,100)

In [71]:
p1 <= p4

False

In [72]:
p1>=p4

False

In [73]:
from functools import total_ordering

In [75]:
@total_ordering
class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    
    def __abs__(self):
        return sqrt(self.x**2 +self.y**2)
    
    def __repr__(self):
        return f'Point({self.x},{self.y})'
    
    def __eq_(self,other):
        if isinstance(other,Point):
            return self.x == other.x and self.y == other.y
        else:
            return False
        
    def __lt__(self,other):
        if isinstance(other,Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented

In [76]:
p1,p2,p3,p4 = Point(2,3),Point(2,3),Point(0,0),Point(100,100)

In [77]:
p1 >= p2

True

In [78]:
p1 <= p4

True